In [1]:
import torch
import torch.utils.data as torch_split
import torch.nn as nn
from torch.nn import MSELoss
import numpy as np
import dataset
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler
from monai.losses import DiceLoss
from monai.losses import FocalLoss
from monai.networks.nets import UNet
import sys
sys.path.insert(1, 'H:/Projects/Kaggle/CZII-CryoET-Object-Identification/preprocessing')
sys.path.insert(1, 'H:/Projects/Kaggle/CZII-CryoET-Object-Identification/postprocessing')
import visual

import metrics

In [2]:
path = "H:/Projects/Kaggle/CZII-CryoET-Object-Identification/datasets/3D/dim104-heat-map-700"
data = dataset.UNetDataset(path=path)

tv_split = 0.8
trn = int(len(data) * tv_split)
val = len(data) - trn

# train_dataset, val_dataset = torch_split.random_split(data, [trn, val])

train_dataset = dataset.UNetDataset(path=path, train=True)
val_dataset = dataset.UNetDataset(path=path, val=True)

labels = [
"background",
"apo-ferritin(E)",
"beta-amylase(NS)",
"beta-galactosidase(H)",
"ribosome(E)",
"thyroglobulin(H)",
"virus-like-particle(E)"
]

In [3]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from optuna.pruners import MedianPruner

# Define the objective function for Optuna
def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # ------------------------------ #
    #        HYPERPARAMETERS         #
    # ------------------------------ #
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    decay = trial.suggest_float('decay', 0.3, 1.0)
    # dropout = trial.suggest_float("dropout", 0.25, 0.5)
    dropout = 0.3
    # regularization_strength = trial.suggest_float("regularization_strength", 1e-4, 1e-2, log=True)
    # alpha = trial.suggest_float("alpha", 0.25, 1.0)
    # theta = trial.suggest_float("theta", 0.1, 0.9)
    # theta = 0.6
    # gamma = trial.suggest_float("gamma", 2.0, 5.0)
    
    # Model initialization
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=7,
        channels=(64, 128, 256, 512),
        strides=(2, 2, 2),
        num_res_units=2,
        dropout=dropout,
    ).to(device)
    
    num_epochs = 30
    batch_size = 16

    # ------------------------------ #
    #        TRAINING METHODS        #
    # ------------------------------ #
    weights = torch.tensor([0.0434743, 1.16546, 1.1661, 1.16513, 1.14281, 1.15554, 1.16149]).to(device)  # Example weights for classes

    # dice_loss = DiceLoss(to_onehot_y=False, softmax=True, weight=weights).to(device)
    # focal_loss = FocalLoss(to_onehot_y=False, use_softmax=True, weight=weights, gamma=gamma ).to(device)
    mse_loss = nn.MSELoss(reduction='none')
    
    optimizer = Adam(model.parameters(), lr=lr)

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=decay)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=False, num_workers=4)

    def add_regularization_loss(model, regularization_type, regularization_strength):
        reg_loss = 0
        for param in model.parameters():
            reg_loss += torch.sum(param ** 2)
        return regularization_strength * reg_loss

    # ------------------------------ #
    #             TRAIN              #
    # ------------------------------ #
    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            input, target = batch['src'].to(device), batch['tgt'].to(device)
            optimizer.zero_grad()
            output = model(input)
            # loss = (theta) * dice_loss(outputs, targets) + (1 - theta) * focal_loss(outputs, targets)
            indv_loss = mse_loss(output, target)
            weighted_loss = (indv_loss * weights.view(1, -1, 1, 1, 1)).mean()
            loss = weighted_loss
            
            # reg_loss = add_regularization_loss(model, "L2", regularization_strength)

            loss.backward()
            optimizer.step()

        scheduler.step()
            
        # ---------- #
        # VALIDATION #
        # ---------- #
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input, target = batch['src'].float().to(device), batch['tgt'].long().to(device)
                output = model(input)
                # loss = (theta) * dice_loss(outputs, targets) + (1 - theta) * focal_loss(outputs, targets)
                loss = mse_loss(output, target)
                weighted_loss = (loss * weights.view(1, -1, 1, 1, 1)).mean()
                val_loss += weighted_loss.item()
                
        # print("batch done")
        val_loss /= len(val_loader)
        
        trial.report(val_loss, epoch)
        print(f"Epoch {epoch} loss: {val_loss}")
        
        if trial.should_prune():
            
            raise optuna.exceptions.TrialPruned()

    return val_loss

n_epochs = 25

study = optuna.create_study(direction="minimize", pruner=MedianPruner(n_startup_trials=4, n_warmup_steps=25))
study.optimize(objective, n_trials=15)

print("Best hyperparameters:", study.best_params)

[I 2024-12-30 08:17:37,592] A new study created in memory with name: no-name-b97bc9c1-8be6-4593-ad7a-82cef1a83e25


Epoch 0 loss: 0.19253355099095237
Epoch 1 loss: 0.11746144874228372
Epoch 2 loss: 0.0855516286359893
Epoch 3 loss: 0.06825231264034907
Epoch 4 loss: 0.058345901055468455
Epoch 5 loss: 0.05078086960646841
Epoch 6 loss: 0.04474914198120435
Epoch 7 loss: 0.039943107300334506
Epoch 8 loss: 0.03659689881735378
Epoch 9 loss: 0.03378640198045307
Epoch 10 loss: 0.0314084357685513
Epoch 11 loss: 0.0293273131052653
Epoch 12 loss: 0.027723345905542374
Epoch 13 loss: 0.026302766882710986
Epoch 14 loss: 0.02498713756601016
Epoch 15 loss: 0.023873560337556735
Epoch 16 loss: 0.022953530980481043
Epoch 17 loss: 0.0220837342656321
Epoch 18 loss: 0.021223079413175583
Epoch 19 loss: 0.020515827875998285
Epoch 20 loss: 0.019931494982706174
Epoch 21 loss: 0.019347550968329113
Epoch 22 loss: 0.018852649049626455
Epoch 23 loss: 0.01835367062853442
Epoch 24 loss: 0.01790864910516474
Epoch 25 loss: 0.017505693559845287
Epoch 26 loss: 0.017096587353282504
Epoch 27 loss: 0.01678870576951239
Epoch 28 loss: 0.0164

[I 2024-12-30 08:52:50,073] Trial 0 finished with value: 0.016113667231467035 and parameters: {'lr': 1.985179111620316e-05, 'decay': 0.8311906302523631}. Best is trial 0 with value: 0.016113667231467035.


Epoch 29 loss: 0.016113667231467035
Epoch 0 loss: 0.12776863078276315
Epoch 1 loss: 0.0702606663107872
Epoch 2 loss: 0.0479134540590975
Epoch 3 loss: 0.03514339733454916
Epoch 4 loss: 0.02781997538275189
Epoch 5 loss: 0.022498129556576412
Epoch 6 loss: 0.018812078775631055
Epoch 7 loss: 0.016030598017904494
Epoch 8 loss: 0.014188543893396854
Epoch 9 loss: 0.012756804728673564
Epoch 10 loss: 0.011563996163507303
Epoch 11 loss: 0.010616312631302409
Epoch 12 loss: 0.009907609472672144
Epoch 13 loss: 0.009328823847075304
Epoch 14 loss: 0.00882801247967614
Epoch 15 loss: 0.008394705545571115
Epoch 16 loss: 0.007999508020778498
Epoch 17 loss: 0.007585719796932406
Epoch 18 loss: 0.007507151717113124
Epoch 19 loss: 0.0072546713054180145
Epoch 20 loss: 0.0070174927823245525
Epoch 21 loss: 0.006842651663141118
Epoch 22 loss: 0.00668547638795442
Epoch 23 loss: 0.006527445072101222
Epoch 24 loss: 0.006465236863328351
Epoch 25 loss: 0.00630714138969779
Epoch 26 loss: 0.006137951225456264
Epoch 27 l

[I 2024-12-30 09:33:57,601] Trial 1 finished with value: 0.005857134858767192 and parameters: {'lr': 5.570803796931761e-05, 'decay': 0.9054139490638864}. Best is trial 1 with value: 0.005857134858767192.


...

Epoch 29 loss: 0.0012202517051870625
Epoch 0 loss: 0.004283553817205959
Epoch 1 loss: 0.0020860470831394196
Epoch 2 loss: 0.0016491394360653227
Epoch 3 loss: 0.00145602409934832
Epoch 4 loss: 0.0013419203460216522
Epoch 5 loss: 0.0013056227083628376
Epoch 6 loss: 0.0015295806547833814
Epoch 7 loss: 0.0012149725161078903
Epoch 8 loss: 0.0012130258449663718
Epoch 9 loss: 0.001493973334112929
Epoch 10 loss: 0.0011917426406095426
Epoch 11 loss: 0.0011895516137075094
Epoch 12 loss: 0.0011746768141165376
Epoch 13 loss: 0.0011893852950177258
Epoch 14 loss: 0.00119099041654004
Epoch 15 loss: 0.0011821709987190035
Epoch 16 loss: 0.0011950746411457658
Epoch 17 loss: 0.0011643320839438173
Epoch 18 loss: 0.0011648871263282166
Epoch 19 loss: 0.0011714657044245137
Epoch 20 loss: 0.0011688937407193913
Epoch 21 loss: 0.0011638964836796124
Epoch 22 loss: 0.0011962611719758974
Epoch 23 loss: 0.0011593359134470422
Epoch 24 loss: 0.0011825534877263838
Epoch 25 loss: 0.001156240649935272
Epoch 26 loss: 0.0

[I 2024-12-30 17:09:45,824] Trial 13 finished with value: 0.0011657905350956651 and parameters: {'lr': 0.0026096063451923196, 'decay': 0.5497714600593803}. Best is trial 9 with value: 0.0011574472818109724.


Epoch 29 loss: 0.0011657905350956651
Epoch 0 loss: 0.004487482365220785
Epoch 1 loss: 0.001796971993624336
Epoch 2 loss: 0.0016902048502945239
Epoch 3 loss: 0.00145911466744211
Epoch 4 loss: 0.001348349201079044
Epoch 5 loss: 0.0013644146965816617
Epoch 6 loss: 0.0014046015373120706
Epoch 7 loss: 0.0012314291282867391
Epoch 8 loss: 0.0011510178188069
Epoch 9 loss: 0.0012408006134339506
Epoch 10 loss: 0.0011759605258703232
Epoch 11 loss: 0.0011675529337177675
Epoch 12 loss: 0.001213257330366307
Epoch 13 loss: 0.0011952105236964093
Epoch 14 loss: 0.0011354746012431052
Epoch 15 loss: 0.0011834693788033393
Epoch 16 loss: 0.0011553307219098012
Epoch 17 loss: 0.001131662762620383
Epoch 18 loss: 0.0013113769236952066
Epoch 19 loss: 0.0010872208772020207
Epoch 20 loss: 0.0011378138895250028
Epoch 21 loss: 0.0015075804096543128
Epoch 22 loss: 0.0012620626690073146
Epoch 23 loss: 0.0011597041221749452
Epoch 24 loss: 0.0011706815437517231
Epoch 25 loss: 0.0011806942315565215
Epoch 26 loss: 0.0011

[I 2024-12-30 17:39:10,991] Trial 14 finished with value: 0.0011473140912130475 and parameters: {'lr': 0.0031310830139417182, 'decay': 0.8847056169377736}. Best is trial 14 with value: 0.0011473140912130475.


Epoch 29 loss: 0.0011473140912130475
Best hyperparameters: {'lr': 0.0031310830139417182, 'decay': 0.8847056169377736}


In [3]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from optuna.pruners import MedianPruner

vis = visual.loss_precision_recall(20, labels, 2.0)
vis.start()
vis.new_trial()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------------------ #
#        HYPERPARAMETERS         #
# ------------------------------ #
lr = 5.0e-4
decay = 0.9
# dropout = trial.suggest_float("dropout", 0.25, 0.5)
dropout = 0.3
regularization_strength = 1e-3
# alpha = trial.suggest_float("alpha", 0.25, 1.0)
# theta = trial.suggest_float("theta", 0.1, 0.9)
theta = 0.5
# gamma = trial.suggest_float("gamma", 2.0, 5.0)
gamma = 4.0
# Model initialization
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=7,
    channels=(64, 128, 256, 512),
    strides=(2, 2, 2),
    num_res_units=2,
    dropout=dropout,
).to(device)

num_epochs = 15
batch_size = 16

# ------------------------------ #
#        TRAINING METHODS        #
# ------------------------------ #
weights = torch.tensor([0.0434743, 1.16546, 1.1661, 1.16513, 1.14281, 1.15554, 1.16149]).to(device)  # Example weights for classes
# weights = torch.tensor([1.0,1.0,1.0,1.0,1.0,1.0,1.0])
dice_loss = DiceLoss(to_onehot_y=False, softmax=True, weight=weights).to(device)
focal_loss = FocalLoss(to_onehot_y=False, use_softmax=True, weight=weights, gamma=gamma ).to(device)
# mse_loss = nn.MSELoss(reduction='none')

optimizer = Adam(model.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=decay)

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=False, num_workers=4)

def add_regularization_loss(model, regularization_strength):
    reg_loss = 0
    for param in model.parameters():
        reg_loss += torch.sum(param ** 2)
    return regularization_strength * reg_loss

# ------------------------------ #
#             TRAIN              #
# ------------------------------ #
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input, target = batch['src'].to(device), batch['tgt'].to(device)
        optimizer.zero_grad()
        output = model(input)
        loss = (theta) * dice_loss(output, target) + (1 - theta) * focal_loss(output, target)
        reg_loss = add_regularization_loss(model, regularization_strength)
        loss += reg_loss
        loss.backward()
        optimizer.step()

    scheduler.step()
        
    # ---------- #
    # VALIDATION #
    # ---------- #
    model.eval()
    val_loss = 0
    precision = torch.zeros((7))
    recall = torch.zeros((7))
    with torch.no_grad():
        for batch in val_loader:
            input, target = batch['src'].float().to(device), batch['tgt'].long().to(device)
            output = model(input)
            loss = (theta) * dice_loss(output, target) + (1 - theta) * focal_loss(output, target)
            reg_loss = add_regularization_loss(model, regularization_strength)
            loss += reg_loss
            val_loss += loss.item()
            p, r = metrics.continuous_precision_recall(target.to('cpu'), torch.softmax(output.to('cpu'), dim=1))
            precision += p
            recall += r
    val_loss /= len(val_loader)
    pr = torch.stack([precision, recall], dim=0)
    vis.report(val_loss, pr)
            
    print(f"Epoch {epoch} loss: {val_loss}")
    

Epoch 0 loss: 0.6432971689436171
Epoch 1 loss: 0.5598176187939115
Epoch 2 loss: 0.5357193417019315
Epoch 3 loss: 0.5257072117593553
Epoch 4 loss: 0.525143735938602


KeyboardInterrupt: 

In [4]:
torch.save(model.state_dict(), "HeatNet-1-0.pth")


In [1]:
# Inference
import sys
sys.path.insert(1, 'H:/Projects/Kaggle/CZII-CryoET-Object-Identification/preprocessing')
import load
import augment
import os
import torch
from monai.networks.nets import UNet
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
run = 'TS_6_4'
root = load.get_root()
picks = load.get_picks_dict(root)
vol, coords, scales = load.get_run_volume_picks(root, run=run, level=0)
for key in coords.keys():
    coords[key] = np.array(coords[key], dtype=np.int16)
coord_list = []
for key in coords.keys():
    coord_list.append(coords[key])
radii = [ 6,
          6,
          9,
          15,
          13,
          14 ]
params = augment.aug_params
params["final_size"] = (104,104,104)
params["flip_prob"] = 0.0
params["patch_size"] = (104,104,104)
params["rot_prob"] = 0.0

In [5]:
mask = load.create_exponential_heatmap_gpu(6, vol.shape, coord_list, radii).cpu().numpy()

In [35]:
sample = augment.random_augmentation_gpu(vol, 
                                mask,
                                num_samples=1, 
                                aug_params=params
                                )

src = sample[0]["source"].unsqueeze(0).to(device)
tgt = sample[0]["target"].unsqueeze(0).to(device)

model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=7,
    channels=(64, 128, 256, 512),
    strides=(2, 2, 2),
    num_res_units=2,
    dropout=0.1,
).to(device)
model.load_state_dict(torch.load("HeatNet-1-0.pth"))

model.eval()
pred = torch.softmax(model(src), dim=1).to('cpu')
pred = pred.squeeze().to('cpu').detach().numpy()
src = src.to('cpu').squeeze()
tgt = tgt.squeeze(0).to('cpu').detach().numpy()
print(f"src shape {src.shape}")
print(f"tgt shape {tgt.shape}")
print(f"pred shape {pred.shape}")


src shape torch.Size([104, 104, 104])
tgt shape (7, 104, 104, 104)
pred shape (7, 104, 104, 104)


In [36]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

def plot_cross_section(i):
    vol1 = 1.0 - src
    vol1 = np.zeros(pred[0].shape)
    vol2 = pred[1]
    
    plt.figure(figsize=(15, 5))
    alpha = 0.3

    # Slice at x-coordinate
    plt.subplot(131)
    plt.imshow(vol1[i, :, :], cmap="viridis", alpha=alpha)
    plt.imshow(vol2[i, :, :], cmap="Reds", alpha=alpha)  # Overlay mask with transparency
    plt.title(f'Slice at x={i}')

    # Slice at y-coordinate
    plt.subplot(132)
    plt.imshow(vol1[:, i, :], cmap="viridis", alpha=alpha)
    plt.imshow(vol2[:, i, :], cmap="Blues", alpha=alpha)
    plt.title(f'Slice at y={i}')

    # Slice at z-coordinate
    plt.subplot(133)
    plt.imshow(vol1[:, :, i], cmap="viridis", alpha=alpha)
    plt.imshow(vol2[:, :, i], cmap="Reds", alpha=alpha)
    plt.title(f'Slice at z={i}')

    plt.show()

# Interactive Slider for scrolling through slices
interact(plot_cross_section, i=(0, src.shape[0] - 1))

interactive(children=(IntSlider(value=51, description='i', max=103), Output()), _dom_classes=('widget-interact…

<function __main__.plot_cross_section(i)>